# Introduction

### Project initialization and setup

Importing all of the libraries that will be used. In the project.

In [20]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


Display options (make this clearer)

In [21]:

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)

Explain what this is

In [22]:
DB_PATH = "viewer_interactions.db"

try:
    conn = sqlite3.connect(DB_PATH)
    print("Connected successfully!")
except sqlite3.Error as e:
    print("Connection failed:", e)

Connected successfully!


Listing all the tables

In [23]:
tables_query = """
               SELECT name
               FROM sqlite_master
               WHERE type='table'
               ORDER BY name; \
               """

tables_df = pd.read_sql_query(tables_query, conn)
print("Tables in the database:")
display(tables_df)

Tables in the database:


,name


In [24]:
table_names = tables_df["name"].tolist()

schemas = {}

for table in table_names:
    pragma_query = f"PRAGMA table_info({table});"
    schema_df = pd.read_sql_query(pragma_query, conn)
    schemas[table] = schema_df
    print(f"\nSchema for table '{table}':")
    display(schema_df)

Creating a dictionary of type table_name -> DataFrame

In [25]:
# dfs = {
#    "interactions": DataFrame with columns [user_id, movie_id, rating, timestamp, ...],
#    "movies":       DataFrame with columns [movie_id, title, genres, year, ...],
#    "users":        DataFrame with columns [user_id, age, country, ...]
# }

Data frame shape where shape is the number of rows and the second number is the number of columns. We are specifically grabbing the names of the sets of tables

In [26]:
dfs = {}

for t in table_names:
    df = pd.read_sql_query(f"SELECT * FROM {t};", conn)
    dfs[t] = df
    print(f"\nLoaded table '{t}' with shape {df.shape}")

Example of using the dfs dictionary

In [27]:
def search_by_parameter(table_name, key, value):
    df = dfs[table_name]

    if value is None:
        return df[df[key].isna()]

    return df[df[key] == value]

Giacomo thing

In [28]:
tables = pd.read_sql(
    "SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';",
    conn
    )['name'].tolist()

print("=== DATA DICTIONARY ===\n")

for table in tables:
    print(f"Table: {table}")
    print("-" * (7 + len(table)))

    # Get actual column info from PRAGMA but filter to nice output
    schema = pd.read_sql(f"PRAGMA table_info('{table}')", conn)

    # Keep only real schema fields you want (remove cid, default, pk if desired)
    clean_schema = schema[['name', 'type', ]]

    print(clean_schema.to_string(index=False))
    print("\n")

=== DATA DICTIONARY ===



In [29]:
for name, df in dfs.items():
    print(f"\n{name} missing values (%):")
    missing_pct = df.isna().mean() * 100
    display(missing_pct.to_frame("missing_%"))

Counting all missing values, diagnostics purposes only

A function to calculate missing std. ratings of films

In [30]:
def compute_film_std(df):
    film_stats = (
        df.groupby('movie_id')['rating']
        .apply(list)
        .reset_index(name='ratings')
    )

    def manual_std(ratings):
        ratings = np.array(ratings)
        n = len(ratings)
        if n <= 1:
            return 0.0
        mean = ratings.mean()
        return np.sqrt(((ratings - mean) ** 2).mean())

    film_stats['std_rating'] = film_stats['ratings'].apply(manual_std)

    return film_stats[['movie_id', 'std_rating']]

# Compute std for all films
viewer_ratings = dfs['viewer_ratings']
film_std = compute_film_std(viewer_ratings)

# Load movies_statistics
movies_stats = dfs["movie_statistics"]

# Compute the old percentage before merging
old_null_pct = dfs["movie_statistics"]["std_rating"].isna().mean() * 100

# Merge new std values
movies_stats = movies_stats.merge(
    film_std,
    on="movie_id",
    how="left",
    suffixes=("", "_new")
)

# Replace old std_rating with the new one
movies_stats["std_rating"] = movies_stats["std_rating_new"]
movies_stats.drop(columns=["std_rating_new"], inplace=True)

# Save updated table
dfs["movie_statistics"] = movies_stats

# Compute new percentage ---
new_null_pct = movies_stats["std_rating"].isna().mean() * 100

# absolute improvement (percentage points)
improvement_abs = old_null_pct - new_null_pct

# relative improvement (how many percent of the original NaNs we removed)
improvement_rel = (improvement_abs / old_null_pct) * 100 if old_null_pct > 0 else 0

print(f"Missing values reduced from {old_null_pct:.2f}% to {new_null_pct:.2f}%.")
print(f"Absolute improvement: {improvement_abs:.2f}%")
print(f"Relative improvement: {improvement_rel:.2f}% better than before.")

KeyError: 'viewer_ratings'

In [31]:
#Figure out how to drop na values in general
movie_stats = dfs['movie_statistics']
#print(movie_stats[movie_stats['std_rating'] == pd.isnull(movie_stats['std_rating'])])
#movie_stats = movie_stats[movie_stats['std_rating'].notna()]
#print(movie_stats)

print(f"Before cleaning: {len(movie_stats)} movies")
movie_stats = movie_stats.dropna(subset=['std_rating'])
dfs['movie_statistics'] = movie_stats
print(f"After removing single-rating movies: {len(movie_stats)} movies")

KeyError: 'movie_statistics'

Calculating the missing total_ratings of movies

In [ ]:
movie_stats = dfs["movie_statistics"]
viewer_ratings = dfs["viewer_ratings"]

# Collecting all the movies with absent total_rating in a dictionary
missing_dict = {}

missing = search_by_parameter('movie_statistics', 'total_ratings', None)
missing_dict = {row.movie_id: 0 for row in missing.itertuples(index=False)}

# Iterating through viewer_ratings and manually counting the ratings for each film
for row in viewer_ratings.itertuples(index=False):
    movie_id = row.movie_id
    if movie_id in missing_dict:
        missing_dict[movie_id] += 1

# Update movie_stats
for row in movie_stats.itertuples(index=True):
    if row.movie_id in missing_dict:
        movie_stats.at[row.Index, "total_ratings"] = missing_dict[row.movie_id]

dfs["movie_statistics"] = movie_stats

Calculating Missing Averages

In [ ]:
# Finding movies with null avg_rating
missing_avg = search_by_parameter('movie_statistics', 'avg_rating', None)

# Creating a dict of type { movie_id : avg_rating }
# set 0 as base value for now, might change it later
missing_avg_dict = {row.movie_id: 0 for row in missing_avg.itertuples(index=False)}

# Storing the sum of all ratings for each movie
rating_sums = {movie_id: 0 for movie_id in missing_avg_dict}

# Iterating through viewer ratings and adding to sum if movie_id matches
for row in viewer_ratings.itertuples(index=False):
    movie_id = row.movie_id
    rating = row.rating

    if movie_id in missing_avg_dict:
        rating_sums[movie_id] += rating

for row in movie_statistics.itertuples():
    movie_id = row.movie_id

    if movie_id in rating_sums:
        total = row.total_ratings # I'm assuming that my calculations of total_ratings per movie is correct ang i got rid of                            all null values

        # IF FORE SOME MAGICAL REASON THERE IS STILL A NULL THEN IGNORE
        if pd.isna(total) or total == 0:
            avg = 0
        else:
            avg = rating_sums[movie_id] / total

        movie_stats.at[row.Index, "avg_rating"] = avg

Calculating the missing min and max ratings for movies

In [ ]:
movie_stats = dfs["movie_statistics"]
viewer_ratings = dfs["viewer_ratings"]

# Find movies with missing min_rating and max_rating using your function
missing_min = search_by_parameter('movie_statistics', 'min_rating', None)
missing_max = search_by_parameter('movie_statistics', 'max_rating', None)

# Combine them as some movies may be in both
missing_ids = set(missing_min["movie_id"]) | set(missing_max["movie_id"])

# Take only ratings for the movies we care
relevant_ratings = viewer_ratings[viewer_ratings["movie_id"].isin(missing_ids)]

# Building a nested dict {movie_id : {"min": ..., "max": ...}}
min_max_dict = {}

for row in relevant_ratings.itertuples(index=False):
    movie_id = row.movie_id
    rating = row.rating

    if movie_id not in min_max_dict:
        min_max_dict[movie_id] = {"min": rating, "max": rating}
    else:
        if rating < min_max_dict[movie_id]["min"]:
            min_max_dict[movie_id]["min"] = rating
        if rating > min_max_dict[movie_id]["max"]:
            min_max_dict[movie_id]["max"] = rating

# Update movie_statistics
for row in movie_stats.itertuples(index=True):
    movie_id = row.movie_id

    if movie_id in min_max_dict:
        if pd.isna(row.min_rating):
            movie_stats.at[row.Index, "min_rating"] = min_max_dict[movie_id]["min"]
        if pd.isna(row.max_rating):
            movie_stats.at[row.Index, "max_rating"] = min_max_dict[movie_id]["max"]

dfs["movie_statistics"] = movie_stats

Finding missing unique users

In [ ]:
movie_stats = dfs["movie_statistics"]
viewer_ratings = dfs["viewer_ratings"]

missing_unique = search_by_parameter('movie_statistics', 'unique_users', None)

# creating my favorite movie set
missing_movie_ids = {row.movie_id for row in missing_unique.itertuples(index=False)}

print(f"Фильмов с NULL unique_users: {len(missing_movie_ids)}")

# creating a dict movie_id: customer_id
unique_users_dict = {movie_id: set() for movie_id in missing_movie_ids}

# Gathering unique users
for row in viewer_ratings.itertuples(index=False):
    movie_id = row.movie_id

    # as always im getting only those movies which have null for unique users
    if movie_id in unique_users_dict:
        unique_users_dict[movie_id].add(row.customer_id)

# Counting unique users
updated = 0
for movie_id, users in unique_users_dict.items():
    count = len(users)  # unique users count

    movie_stats.loc[
        movie_stats["movie_id"] == movie_id,
        "unique_users"
    ] = count

    updated += 1

# just in case if a movie has 0 ratings im setting unique users to 0
movie_stats["unique_users"] = movie_stats["unique_users"].fillna(0).astype(int)

# Updating
dfs["movie_statistics"] = movie_stats

Checking for any duplicate movie ids and removing them from all datasets to clean the data.

In [ ]:
# this must run before other code blocks
rating_data = pd.read_sql("SELECT * FROM viewer_ratings", conn)
user_data = pd.read_sql("SELECT * FROM user_statistics", conn)
movie_data = pd.read_sql("SELECT * FROM movies", conn)
movie_statistics = pd.read_sql("SELECT * FROM movie_statistics", conn)


Merging the Movies and movie statis filling in missing values on either dataset and converting all of the dates to type DateTime as well as all counts and years to integers.
This is in order to clean our movie data before merging it with our user data to fill in any recoverable missing values.

In [ ]:

# this will print out the duplicate movie_ids (empty if none)
bad_ids_movies =  movie_data[movie_data['movie_id'].duplicated (keep = False)]['movie_id'].unique().tolist()
bad_ids_stats =  movie_statistics[movie_statistics['movie_id'].duplicated (keep = False)]['movie_id'].unique().tolist()
bad_ids = list(set(bad_ids_movies + bad_ids_stats))
print('bad movie ids to remove: ', bad_ids)

# now these bad_ids will be removed from all of the data
rating_data = rating_data[~rating_data['movie_id'].isin(bad_ids)]
movie_data = movie_data[~movie_data['movie_id'].isin(bad_ids)]
movie_statistics = movie_statistics[~movie_statistics['movie_id'].isin(bad_ids)]


# changing types to int and datetime
movie_data['year_of_release'] = movie_data['year_of_release'].astype('Int64')
movie_statistics['total_ratings'] = movie_statistics['total_ratings'].astype('Int64')
movie_statistics['unique_users'] = movie_statistics['unique_users'].astype('Int64')
movie_statistics['year_of_release'] = movie_statistics['year_of_release'].astype('Int64')
movie_statistics['first_rating_date'] = pd.to_datetime(movie_statistics['first_rating_date'], errors = 'coerce')
movie_statistics['last_rating_date'] = pd.to_datetime(movie_statistics['last_rating_date'], errors = 'coerce')

# merging the two datasets
movie_full = movie_data.merge(movie_statistics, on = 'movie_id', how = 'outer', suffixes = ('_movies', '_stats'))
# filling in missing values on either dataset to create a full movie dataset
movie_full['title'] = movie_full['title_movies'].combine_first(movie_full['title_stats'])
movie_full['year_of_release'] = movie_full['year_of_release_movies'].combine_first(movie_full['year_of_release_stats'])
# dropping the temporary movies and stats columns
movie_full = movie_full.drop(columns = ['title_movies', 'title_stats', 'year_of_release_movies', 'year_of_release_stats'])
dfs['movie_full'] = movie_full

bad movie ids to remove:  [21756, 19549]


- Converts the date parameter in viewer_ratings to datetime.
- Merges viewer_ratings, movies, movie_statistics and user_statistics into one dataset as merged_data.

In [ ]:



rating_data['date'] = pd.to_datetime(rating_data['date'], errors = 'coerce')
rating_data.dtypes
# movie_statistics['first_rating_date'] = 
movie_statistics['total_ratings'] = movie_statistics['total_ratings'].astype('Int64')
movie_statistics['first_rating_date'] = pd.to_datetime(movie_statistics['first_rating_date'], errors = 'coerce')



merged_data = rating_data.merge(movie_data, on = 'movie_id', how = 'left')
merged_data = merged_data.merge(user_data, on = 'customer_id', how = 'left')
merged_data = merged_data.merge(movie_statistics, on = 'movie_id', how = 'left')
# avg rating standard rating mean rating
'''
want to merge
- title
- year of release
want to keep independent
- avg rating
- std rating
- min rating
- max rating
- first rating date
- last rating date
'''

print("Total columns:", len(merged_data.columns))
list(merged_data.columns)

Giacomo

Ryder

In [ ]:
import seaborn as sns


# Markdown cell above
"""
### Global Rating Distribution

There is a clear over positive bias. This can be due to the fact that this was done on a streaming platform where users are more likely to rate higher
"""

plt.figure(figsize=(11, 6)) # made it this size for a better fit in the read me
ax = sns.histplot(
    data=dfs['viewer_ratings'],
    x='rating',
    bins=5,
    discrete=True, #tells seaborn to treat x as integer values
    color="#0062ff",
    edgecolor='white',
    alpha=0.85,  #transparency
    linewidth=1.5
)

# Add exact counts on top of each bar
for rect in ax.patches:
    height = rect.get_height()
    ax.text(
        rect.get_x() + rect.get_width()/2., 
        height + 200_000,                    # a bit above the bar
        f'{int(height):,}',                  # adds commas: 12,345,678
        ha='center', va='bottom', fontsize=12, fontweight='bold'
    )

# Clean y-axis
plt.ylabel('Number of Ratings (millions)', fontsize=12)
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x/1e6:.1f}M'))

plt.title('Distribution of Viewer Ratings\n(Strong Positive Bias)', 
          fontsize=15, pad=20)
plt.xlabel('Rating (1–5)', fontsize=12)
plt.xticks(range(0, 7))
plt.ylim(0, 13_500_000)
plt.show()

In [ ]:
#test code block to commit